## Check all past votes in last 7 days for frequent authors

In [8]:
import pandas as pd
from datetime import datetime, timedelta
from steevebase.io import mongo_factory
from voter.config import CONFIG

mongo_address = CONFIG['DATABASE']['ADDRESS']
db = mongo_factory(mongo_address)

In [9]:
start_time = datetime.utcnow() - timedelta(days=7)
query = {
    'vote_time': {
        '$gte': start_time
    }
}

past_votes_col = db.get_collection("past_votes")
df_past_votes = pd.DataFrame([x for x in past_votes_col.find(query)])

In [10]:
df_past_votes.shape

(5308, 5)

In [ ]:
query = {
    'created': {
        '$gte': start_time
    }
}
df_past_voted_posts = pd.DataFrame([x for x in raw_posts_col.find(query)])

In [ ]:
df_past_voted_posts[df_past_voted_posts['_id'].isin(df_past_votes['_id'])].shape

In [ ]:
df_past_voted_posts[df_past_voted_posts['_id'].isin(df_past_votes['_id'])].created.max()

In [ ]:
grps = df_past_voted_posts[df_past_voted_posts['_id'].isin(df_past_votes['_id'])].groupby('author')
cc = grps.count()
cc['_id'].sort_values(ascending=False)

In [ ]:
(cc['_id'].sort_values(ascending=False) > 7).sum()

In [ ]:
cc['_id'][cc['_id'] > 7].sum()

In [ ]:
import matplotlib.pyplot as plt

%matplotlib notebook

plt.figure()
cc['_id'].sort_values(ascending=False).iloc[1:].plot()

plt.savefig('x_normal.png')

In [ ]:
import matplotlib.pyplot as plt

%matplotlib notebook

plt.figure()
plt.tight_layout()
cc['_id'].sort_values(ascending=False).iloc[1:].plot()

plt.savefig('x_tight.png', bbox_inches='tight')

In [ ]:
def check_author_not_exploiting(post):
    try:
        author = post['author']
    except PostDoesNotExist:
        return False

    # find out how many votes the author received in the past week
    start_time = datetime.utcnow() - timedelta(days=7)
    last_week = {'$gte': start_time}
    query = {'author': author, 'created': last_week}
    authors_posts, _ = load_data(db_address, 'raw_posts', query)
    count = sum([1 for x in authors_posts if 'hr1' in [y['voter'] for y in x['active_votes']]])
    print(count)

#         return count < EXPLOIT_LIMIT
    return count < 7

In [ ]:
check_author_not_exploiting(Post('@bosssteemit/starbase-piattaforma-di-crowdfunding-con-token-blockchain-20171228t04730705z'))